In [ ]:
import pandas as pd
import re
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from pandas import DataFrame
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from numpy import dtype
import warnings 
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from scipy import sparse
from sklearn import preprocessing
import random
from random import randint
from sklearn.ensemble import GradientBoostingClassifier


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Скрипт по анализу текста с учетом прочих фичей (возраст, сумма покупки) на итоговую лояльность клиента**

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# file_folder = 'Issue_MOCK_Data_v0 - Sheet1.csv'
df= pd.read_csv('/content/drive/My Drive/file_name2.csv', low_memory=False)


In [ ]:
df.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 0','Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6'] , axis='columns', inplace=True)
df.head(10)


,user_id,LABEL,theme,feedback,ful_feedback,characters
0,1.0,позитивный,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,Vestibulum ante ipsum primis in faucibus orci ...,NaN
1,2.0,нейтральный,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,NaN,NaN
2,3.0,колеблющийся,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,Vivamus vestibulum sagittis sapien. Cum sociis...,ultrices posuere cubilia
3,4.0,колеблющийся,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,In hac habitasse platea dictumst. Maecenas ut ...,NaN
4,5.0,колеблющийся,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,NaN,dui
5,6.0,нейтральный,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,NaN,nunc
6,7.0,нейтральный,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,NaN,nonummy integer
7,8.0,колеблющийся,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,Morbi vel lectus in quam fringilla rhoncus.,ut
8,9.0,позитивный,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,Pellentesque ultrices mattis odio.,sit amet
9,10.0,отрицательный,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,"Integer aliquet, massa id lobortis convallis, ...",sapien


In [ ]:
df = df.drop(['ful_feedback'], axis=1)

In [ ]:
df.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   user_id     565 non-null    float64
 1   LABEL       570 non-null    object 
 2   theme       569 non-null    object 
 3   feedback    569 non-null    object 
 4   characters  318 non-null    object 
dtypes: float64(1), object(4)
memory usage: 22.4+ KB


0. **LABEL** - итоговая лояльность клиента на тестовых данных
1.   **feedback** - обратная связь от клиента (собственно его отзыв)
2.   **characters** - дополнительные характеристики клиента 
3. **purchase_value** - сумма покупок по клиенту 
4. **ISSUE_TEXT** - общий тест отзыва вместе с темой (конкатенирован)
5. **ISSUE_TEXT** - общий тест отзыва вместе с темой (конкатенирован)




In [ ]:
total_count, total_cols = df.shape
print("Dataframe contains {} observations across {} columns".format(total_count, total_cols))

Dataframe contains 570 observations across 5 columns


In [ ]:
df.groupby('LABEL').size()


LABEL
колеблющийся     145
нейтральный      142
отрицательный    145
позитивный       138
dtype: int64

In [ ]:
df['LABEL'] = df['LABEL'].fillna(value="NOT DEFINED")

In [ ]:
df['user_id'] = df['user_id'].fillna(value=9999)
df['theme'] = df['theme'].fillna(value="нет данных")
df['feedback'] = df['feedback'].fillna(value="нет данных")

In [ ]:
from random import randint 
age = [randint(18,80) for x in range(50)]
purchase_value = [randint(500,5000) for x in range(2000)]
df['characters'] = df['characters'].fillna(value="")
df['age'] = 0
for i in range(len(df['age'])):
  df['age'][i] = random.choice(age)
df['purchase_value'] = 0
for i in range(len(df['purchase_value'])):
  df['purchase_value'][i] = random.choice(purchase_value)



In [ ]:
df.isna().sum()

user_id           0
LABEL             0
theme             0
feedback          0
characters        0
age               0
purchase_value    0
dtype: int64

In [ ]:
df['ISSUE_TEXT'] = df['theme'] + df['feedback']
df.columns

Index(['user_id', 'LABEL', 'theme', 'feedback', 'characters', 'age',
       'purchase_value', 'ISSUE_TEXT'],
      dtype='object')

In [ ]:
df.head(10)

,user_id,LABEL,theme,feedback,characters,age,purchase_value,ISSUE_TEXT
0,1.0,позитивный,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,,37,4734,восхитительный устаревший восхитительный невра...
1,2.0,нейтральный,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,,76,3685,восхитительный устаревший восхитительный невра...
2,3.0,колеблющийся,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,ultrices posuere cubilia,66,3941,восхитительный устаревший восхитительный невра...
3,4.0,колеблющийся,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,,69,4869,восхитительный устаревший восхитительный невра...
4,5.0,колеблющийся,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,dui,59,4163,восхитительный устаревший восхитительный невра...
5,6.0,нейтральный,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,nunc,36,3197,восхитительный устаревший восхитительный невра...
6,7.0,нейтральный,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,nonummy integer,31,1056,восхитительный устаревший восхитительный невра...
7,8.0,колеблющийся,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,ut,26,929,восхитительный устаревший восхитительный невра...
8,9.0,позитивный,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,sit amet,57,1956,восхитительный устаревший восхитительный невра...
9,10.0,отрицательный,восхитительный устаревший восхитительный невра...,тщедушный круто ужасный современный от назойли...,sapien,67,4402,восхитительный устаревший восхитительный невра...


In [ ]:
encoded_labels = pd.get_dummies(df['LABEL'])

In [ ]:
df = pd.concat([df, encoded_labels], axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user_id         570 non-null    float64
 1   LABEL           570 non-null    object 
 2   theme           570 non-null    object 
 3   feedback        570 non-null    object 
 4   characters      570 non-null    object 
 5   age             570 non-null    int64  
 6   purchase_value  570 non-null    int64  
 7   ISSUE_TEXT      570 non-null    object 
 8   колеблющийся    570 non-null    uint8  
 9   нейтральный     570 non-null    uint8  
 10  отрицательный   570 non-null    uint8  
 11  позитивный      570 non-null    uint8  
dtypes: float64(1), int64(2), object(5), uint8(4)
memory usage: 38.0+ KB


### Defining Text Pre-Processing

In [ ]:

stop_words = set(stopwords.words('russian'))

def concatenate_list_data(lst):
    result = ''
    for element in lst:
        result += str(element)
        result += ' '
    return result

def pre_process(text, legacy):

    filtered_words = ['']*len(text)
    stemmer = WordNetLemmatizer
    document = str(text)
    text.strip("/")

    document = re.sub(r'\W', ' ', document)
    document = re.sub("_", "", document)
    document = re.sub("-", "", document)
    document = ''.join([i for i in document if not i.isdigit()])
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()

    word_tokens = word_tokenize(document)
    filtered_words = [w for w in word_tokens if not w in stop_words]
    document = concatenate_list_data(filtered_words)


    return document

In [ ]:
text = 'Не нравится ничего мне в вашем приложении 50 - уйду от Вас'

legacy = ''


text.strip("/")
legacy.strip("/")

document = re.sub(legacy, '', str(text))

document = re.sub(r'\W', ' ', document)
document = re.sub("_", "", document)
document = re.sub("-", "", document)
document = ''.join([i for i in document if not i.isdigit()])
document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)
document = re.sub(r'\s+', ' ', document, flags=re.I)
document = re.sub(r'^b\s+', '', document)
document = document.lower()

word_tokens = word_tokenize(document)
filtered_words = [w for w in word_tokens if not w in stop_words]
document = concatenate_list_data(filtered_words)


print(document)

нравится вашем приложении уйду 


In [ ]:

pre_processed_text = ['']*len(df)
for i in range(df.user_id.count()):
    pre_processed_text[i] = pre_process(df.ISSUE_TEXT[i], df.characters[i])
pre_processed_text[0]

'восхитительный устаревший восхитительный невразумительный невразумительный ужасный хороший устаревший cупер хороший скучный круто круто тщедушный назойливый cупер невразумительный скучный круто современный хороший невразумительный ужасный восхитительный cупер хороший хороший восхитительный круто тщедушный тщедушный круто назойливый тщедушный тщедушный cупер невразумительный невразумительный устаревший ужасный восхитительный хороший хороший невразумительный ужасный хороший назойливый современный устаревший устаревший cупер хороший скучный устаревший ужасный круто современный круто восхитительный ужасный восхитительный современный тщедушный тщедушный хороший круто скучный круто ужасный невразумительный хороший современный устаревший хороший тщедушный тщедушный круто назойливый скучный ужасный ужасный невразумительный восхитительный cупер восхитительный устаревший восхитительный восхитительный круто невразумительный восхитительный восхитительный устаревший назойливый хороший тщедушный cу

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(df.LABEL)
le.classes_
normalized_label = le.transform(df.LABEL)


In [ ]:
X = pre_processed_text 


In [ ]:
y = df['позитивный']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split (
                                    X, y, test_size = 0.2, random_state = 42,
                                    stratify = normalized_label)
y_train

18     1
133    0
47     0
555    0
417    1
      ..
111    0
342    0
148    0
477    1
324    0
Name: позитивный, Length: 456, dtype: uint8

In [ ]:
X_train

In [ ]:
tfidf_vect = TfidfVectorizer(max_df = 0.85, stop_words = stopwords.words('russian'))
xtrain_tfidf2 = tfidf_vect.fit_transform(X_train)
xtrain_tfidf = tfidf_vect.fit_transform(X_train).toarray()
xtest_tfidf = tfidf_vect.transform(X_test).toarray()

In [ ]:
type(xtrain_tfidf2)
xtrain_tfidf2 = pd.DataFrame(xtrain_tfidf)
xtest_tfidf2 = pd.DataFrame(xtest_tfidf)

df3 = pd.concat([df[['age', 'purchase_value']][:456], xtrain_tfidf2], axis = 1)
df3.isna().sum()
df2 = pd.concat([df[['age', 'purchase_value']][456:], xtest_tfidf2 ], axis = 1)
xtest_tfidf2  = df2


In [ ]:
xtrain_tfidf.shape, xtest_tfidf.shape

((456, 2), (114, 2))

In [ ]:
sm = SMOTE(random_state = 42)
sm_xtrain_tfidf, sm_train_y = sm.fit_sample(xtrain_tfidf, y_train)

In [ ]:
np.random.seed(42)
best_val_score = 0
best_clf = None
for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    clf_pipeline = Pipeline(
                            [
                                #("vectorizer", TfidfVectorizer(max_df = 0.85, 
                            #                                stop_words = stopwords.words('english'))),
                                ("classifier", clf())
                            ]
                            )
    val_score = cross_val_score(clf_pipeline, sm_xtrain_tfidf, sm_train_y).mean()
    print(clf)
    print(val_score)
    if val_score > best_val_score:
        best_val_score = val_score
        best_clf = clf
    print("\n")
    
print('Best:')
print(best_val_score)
print(best_clf)

<class 'sklearn.linear_model._logistic.LogisticRegression'>
0.4985611510791367


<class 'sklearn.svm._classes.LinearSVC'>
0.4985611510791367


<class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>
0.5


Best:
0.5
<class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>


In [ ]:
best_clf_pipeline = Pipeline(
                            [
                                 ("classifier", best_clf())
                            ]
                        )
best_clf_pipeline.fit(xtrain_tfidf, y_train)
# best_clf_pipeline.fit(xtrain_tfidf, y_test)

Pipeline(memory=None,
         steps=[('classifier',
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=1000, n_iter_no_change=5, n_jobs=None,
                               penalty='l2', power_t=0.5, random_state=None,
                               shuffle=True, tol=0.001, validation_fraction=0.1,
                               verbose=0, warm_start=False))],
         verbose=False)

In [ ]:
Actual_Risk_Type = y_test.tolist()
Predicted_Risk_Type = (best_clf_pipeline.predict(xtest_tfidf))
matches = 0
match_list = []
for i in range (len(xtest_tfidf)):
    if(Actual_Risk_Type[i] == Predicted_Risk_Type[i]):
        matches+=1
        match_list.append(i)
Accuracy = round(matches/len(xtest_tfidf), 2)
print("{} is the model accuracy".format(Accuracy))
print(Actual_Risk_Type)
print(Predicted_Risk_Type)

0.75 is the model accuracy
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(Actual_Risk_Type, Predicted_Risk_Type, digits=3))

              precision    recall  f1-score   support

           0      0.754     1.000     0.860        86
           1      0.000     0.000     0.000        28

    accuracy                          0.754       114
   macro avg      0.377     0.500     0.430       114
weighted avg      0.569     0.754     0.649       114



In [ ]:
match_list
Xt = pd.DataFrame(X_test)

match_list
X = pd.DataFrame(X)

index_list = X.index.tolist()
values_list = X.values.tolist()
values_list = enumerate(values_list)
values_list_t = Xt.values.tolist()
rec_list = [] 
for i in values_list:
  if i[1] in values_list_t:
    rec_list.append(i[0])
recomend_list = []
for i in rec_list:
  recomendation = df.iloc[i]['user_id']
  recomend_list.append(recomendation)
print(f'ИТОГОВЫЙ СПИСОК USER_ID для рекомендации: {recomend_list}')

ИТОГОВЫЙ СПИСОК USER_ID для рекомендации: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 1

In [ ]:
"""Получение весов слов для LABEL "Положительный"""
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = ["This is very strange",
          "This is very nice"]
vectorizer = TfidfVectorizer(
                        use_idf=True, # utiliza o idf como peso, fazendo tf*idf
                        norm=None, # normaliza os vetores
                        smooth_idf=False, #soma 1 ao N e ao ni => idf = ln(N+1 / ni+1)
                        sublinear_tf=False, #tf = 1+ln(tf)
                        binary=False,
                        min_df=1, max_df=1.0, max_features=None,
                        strip_accents='unicode', # retira os acentos
                        ngram_range=(1,1), preprocessor=None,              stop_words=None, tokenizer=None, vocabulary=None
             )
X = vectorizer.fit_transform(X_test)
idf = vectorizer.idf_
print(dict(zip(vectorizer.get_feature_names(), idf)))

{'cупер': 1.0, 'восхитительныи': 1.0, 'круто': 1.0, 'назоиливыи': 1.0, 'невразумительныи': 1.0, 'скучныи': 1.0, 'скучныитщедушныи': 1.0, 'современныи': 1.0, 'тщедушныи': 1.0, 'ужасныи': 1.0, 'устаревшии': 1.0, 'хорошии': 1.0}
